# Selective Capacitated Fair Pickup and Delivery Problem - Part II


## 0.) Introduction & setup

## 1.) Implementing two algorithms

## 2.) Experimenting with different fairness mesaures

### Min-Max Fairness

### Gini coefficient

## 3.) Tuning our algorithms

## 4.) Testing our algortihms

## 5.) Comparing algorithms